In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from nose.tools import *

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Vanilla WoW ninjaloot predictor
# Sources:
# https://github.com/classicdb/database/tree/classic/Full_DB
# https://www.youtube.com/watch?v=6PY3W404Ca4

In [ ]:
# Loading and cleaning our data*

In [ ]:
items = pd.read_csv('data/vanilla_wow_items.csv', sep = ";")

In [ ]:
items.head()

In [ ]:
items.shape

SyntaxError: invalid syntax (<ipython-input-64-f60872265970>, line 1)

,entry,class,subclass,name,displayid,Quality,Flags,BuyCount,BuyPrice,SellPrice,...,Map,BagFamily,ScriptName,DisenchantID,FoodType,minMoneyLoot,maxMoneyLoot,Duration,ExtraFlags,oftenNinjalooted
0,25,2,7,'Worn Shortsword',1542,1,0,1,35,7,...,0,0,'',0,0,0,0,0,0,1
1,35,2,10,'Bent Staff',472,1,0,1,47,9,...,0,0,'',0,0,0,0,0,0,1
2,36,2,4,'Worn Mace',5194,1,0,1,38,7,...,0,0,'',0,0,0,0,0,0,1
3,37,2,0,'Worn Axe',14029,1,0,1,38,7,...,0,0,'',0,0,0,0,0,0,1
4,38,4,0,'Recruit\'s Shirt',9891,1,0,1,1,1,...,0,0,'',0,0,0,0,0,0,1
